In [1]:
import tensorflow.compat.v1 as tf
import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim

import os
import librosa
import numpy as np

# Extract Feature

In [2]:
checkpoint_path = 'check_points/vggish_model.ckpt'
pca_params_path = 'check_points/vggish_pca_params.npz'
audio_path = 'C:/Users/jae/Music/hearing_loss_split_epp'

In [11]:
def vggish_feat_extract(x, sr):
    input_batch = vggish_input.waveform_to_examples(x, sr)
        
    with tf.Graph().as_default(), tf.Session() as sess:
        vggish_slim.define_vggish_slim()
        vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)
        
        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.INPUT_TENSOR_NAME
        )
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.OUTPUT_TENSOR_NAME
        )
        [embedding_batch] = sess.run([embedding_tensor], feed_dict={features_tensor: input_batch})
        
        return embedding_batch
    
def extract_feature_from_files(category='normal'):
    min_len = 100
    max_len = 0
    vggish_feature = np.empty((0, 128))
    for folder in os.listdir(os.path.join(audio_path, category)):
        for audio_file in os.listdir(os.path.join(audio_path, category, folder)):
            #if audio_file.endswith('.wav') and (audio_file.lower() == 'chapter1.wav' or audio_file.lower() == 'chapter2.wav'):
            if audio_file.endswith('.wav') and audio_file.lower() == 'chapter3.wav':
                x, sr = librosa.load(os.path.join(audio_path, category, folder, audio_file), sr=44100, mono=True)
                temp = np.array(vggish_feat_extract(x, sr))
                if audio_file.lower() == 'chapter1.wav':
                    if min_len > temp.shape[0]: min_len = temp.shape[0]
                    if max_len < temp.shape[0]: max_len = temp.shape[0]
                vggish_feature = np.append(vggish_feature, temp, axis=0)
    print(min_len, max_len)
    return vggish_feature

In [12]:
normal_feat = np.array(extract_feature_from_files(category='normal'))
mild_feat = np.array(extract_feature_from_files(category='mild'))
severe_feat = np.array(extract_feature_from_files(category='severe'))

X = np.append(normal_feat, mild_feat, axis=0)
X = np.append(X, severe_feat, axis=0)
Y = np.zeros(shape=(X.shape[0],))
for i in range(len(Y)):
    if i < len(normal_feat): 
        Y[i] = 1
    elif i < len(normal_feat) + len(mild_feat): 
        Y[i] = 2
    else: 
        Y[i] = 3

INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_poi

# SVM Sample Training

In [14]:
# SVM training
import sklearn.svm as svm

C = 10 # SVM regularization parameter
#clf = svm.SVC(kernel="linear", C=C, random_state=2022)
clf = svm.SVC(kernel="rbf", C=C, max_iter=10000)
clf.fit(X, Y)

# confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X)
print(clf.score(X, Y))
confusion_matrix(Y, y_pred)

0.9148351648351648


array([[178,  11,   8],
       [  7, 223,  17],
       [  7,  12, 265]], dtype=int64)

# Unseen Data Test

In [17]:
# unseen data test
TEST_DATA_TYPE = 1
LABEL = 3
LABEL_DICT = {1:'normal', 2:'mild', 3:'severe'}
unseen_audio_path = os.path.join('D:/data set/hearing loss/integrated data/test_data/', LABEL_DICT[LABEL])

x_1, sr = librosa.load(os.path.join(unseen_audio_path, os.listdir(unseen_audio_path)[0], 'chapter1.wav'), sr=44100, mono=True)
x_2, sr = librosa.load(os.path.join(unseen_audio_path, os.listdir(unseen_audio_path)[0], 'chapter2.wav'), sr=44100, mono=True)
x_3, sr = librosa.load(os.path.join(unseen_audio_path, os.listdir(unseen_audio_path)[0], 'chapter3.wav'), sr=44100, mono=True)

if TEST_DATA_TYPE == 0:
    # chapter 1 & 2
    X_test = np.empty((0, 128))
    X_test = np.append(X_test, np.array(vggish_feat_extract(x_1, sr)), axis=0)
    X_test = np.append(X_test, np.array(vggish_feat_extract(x_2, sr)), axis=0)
else:
    # chapter 3
    X_test = np.array(vggish_feat_extract(x_3, sr))

y_pred = clf.predict(X_test)
print(y_pred)
score = 0
for pred in y_pred:
    if pred == LABEL: score += 1
score /= len(y_pred)
print(score)

INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
[2. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3.]
0.8181818181818182


# Simple 5-fold & Leave One Sample Out

In [21]:
# 5-fold cross validation & leave one sample out
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut

cv_results = cross_validate(clf, X, Y, cv=5, return_train_score=True)
print('train score : ', cv_results['train_score'])
print('test score : ', cv_results['test_score'])

loo = LeaveOneOut()
score = 0 
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    clf.fit(X_train, Y_train)
    if clf.predict(X_test) == Y_test: score += 1
score /= len(X)
print('leave one subject out :', score)

train score :  [0.85166994 0.8654224  0.87033399 0.86064769 0.83807655]
test score :  [0.34117647 0.25098039 0.29411765 0.30314961 0.4015748 ]
leave one subject out : 0.6889238020424194


# Leave-one-Subject-Out

In [104]:
# leave on subject out

def extract_feature_from_files(category='normal'):
    feature_len = np.empty((0,))
    vggish_feature = np.empty((0, 128))
    for folder in os.listdir(os.path.join(audio_path, category)):
        for audio_file in os.listdir(os.path.join(audio_path, category, folder)):
            #if audio_file.endswith('.wav') and (audio_file.lower() == 'chapter1.wav' or audio_file.lower() == 'chapter2.wav'):
            if audio_file.endswith('.wav') and audio_file.lower() == 'chapter3.wav':
                x, sr = librosa.load(os.path.join(audio_path, category, folder, audio_file), sr=44100, mono=True)
                temp = np.array(vggish_feat_extract(x, sr))
                vggish_feature = np.append(vggish_feature, temp, axis=0)
        feature_len = np.append(feature_len, [len(vggish_feature)])
    return vggish_feature, feature_len

normal_feat, normal_feat_len = np.array(extract_feature_from_files(category='normal'))
mild_feat, mild_feat_len = np.array(extract_feature_from_files(category='mild'))
severe_feat, severe_feat_len = np.array(extract_feature_from_files(category='severe'))

mild_feat_len += normal_feat_len[-1]
severe_feat_len += mild_feat_len[-1]
total_feat_len = np.append(normal_feat_len, mild_feat_len)
total_feat_len = np.append(total_feat_len, severe_feat_len)
    
X = np.append(normal_feat, mild_feat, axis=0)
X = np.append(X, severe_feat, axis=0)
Y = np.zeros(shape=(X.shape[0],))
for i in range(len(Y)):
    if i < len(normal_feat): 
        Y[i] = 1
    elif i < len(normal_feat) + len(mild_feat): 
        Y[i] = 2
    else: 
        Y[i] = 3

print(X.shape)

INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_poi

<ipython-input-104-a7bb4271a1a2>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  normal_feat, normal_feat_len = np.array(extract_feature_from_files(category='normal'))


INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_poi

<ipython-input-104-a7bb4271a1a2>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mild_feat, mild_feat_len = np.array(extract_feature_from_files(category='mild'))


INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_points/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from check_poi

<ipython-input-104-a7bb4271a1a2>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  severe_feat, severe_feat_len = np.array(extract_feature_from_files(category='severe'))


In [119]:
test_score = 0
train_score = 0
for i in range(len(total_feat_len)):
    index = [j for j in range(len(X))]
    if i >= 1:
        test_idx = [j for j in range(int(total_feat_len[i-1]), int(total_feat_len[i]))]
        train_idx = np.delete(index, test_idx)
        X_test, Y_test = X[test_idx], Y[test_idx]
        X_train, Y_train = X[train_idx], Y[train_idx]
    else:
        X_test, Y_test = X[:int(total_feat_len[i])], Y[:int(total_feat_len[i])]
        X_train, Y_train = X[int(total_feat_len[i]):], Y[int(total_feat_len[i]):]

    C = 1 # SVM regularization parameter
    #clf = svm.SVC(kernel="linear", C=C, random_state=2022)
    clf = svm.SVC(kernel="rbf", C=C, random_state=2022)
    clf.fit(X_train, Y_train)
    
    y_pred = clf.predict(X_test)
    #print(clf.score(X_test, Y_test))
    test_score += clf.score(X_test, Y_test)
    train_score += clf.score(X_train, Y_train)
print('train score :', train_score / len(total_feat_len))
print('test score :', test_score / len(total_feat_len))

train score : 0.6991749320155702
test score : 0.37882080235755383


# 5-fold Cross Validation

In [120]:
test_score = []
train_score = []
NUM_OF_FOLD = 5

for i in range(NUM_OF_FOLD):
    index = [k for k in range(len(X))]
    test_idx = []
    for j in range(len(total_feat_len)):
        if j >= 1:
            seq_len = int(total_feat_len[j]-total_feat_len[j-1])
            temp = [k for k in range(int(total_feat_len[j-1]+seq_len*i*0.2), int(total_feat_len[j-1]+seq_len*(i+1)*0.2))]
            test_idx.extend(temp)
        else:
            seq_len = int(total_feat_len[j])
            temp = [k for k in range(int(seq_len*i*0.2), int(seq_len*(i+1)*0.2))]
            test_idx.extend(temp)
    train_idx = np.delete(index, test_idx)
    
    X_test, Y_test = X[test_idx], Y[test_idx]        
    X_train, Y_train = X[train_idx], Y[train_idx]
    
    C = 1 # SVM regularization parameter
    #clf = svm.SVC(kernel="linear", C=C, random_state=2022)
    clf = svm.SVC(kernel="rbf", C=C, random_state=2022)
    clf.fit(X_train, Y_train)
    
    y_pred = clf.predict(X_test)
    test_score.extend([clf.score(X_test, Y_test)])
    train_score.extend([clf.score(X_train, Y_train)])
print('train score :', train_score)
print('test score :', test_score)
print(np.mean(test_score))

train score : [0.6919275123558485, 0.690068493150685, 0.6735395189003437, 0.6626712328767124, 0.7009009009009008]
test score : [0.5867768595041323, 0.6041666666666666, 0.6575342465753424, 0.6458333333333334, 0.5780346820809249]
0.6144691576320799


In [ ]:
test_score = []
train_score = []
NUM_OF_FOLD = 5

for i in range(NUM_OF_FOLD):
    index = [k for k in range(len(X))]
    test_idx = []
    for j in range(len(total_feat_len)):
        if j >= 1:
            seq_len = int(total_feat_len[j]-total_feat_len[j-1])
            temp = [k for k in range(int(total_feat_len[j-1]+seq_len*i*0.2), int(total_feat_len[j-1]+seq_len*(i+1)*0.2))]
            test_idx.extend(temp)
        else:
            seq_len = int(total_feat_len[j])
            temp = [k for k in range(int(seq_len*i*0.2), int(seq_len*(i+1)*0.2))]
            test_idx.extend(temp)
    train_idx = np.delete(index, test_idx)
    
    X_test, Y_test = X[test_idx], Y[test_idx]        
    X_train, Y_train = X[train_idx], Y[train_idx]
    
    C = 1 # SVM regularization parameter
    #clf = svm.SVC(kernel="linear", C=C, random_state=2022)
    clf = svm.SVC(kernel="rbf", C=C, random_state=2022)
    clf.fit(X_train, Y_train)
    
    y_pred = clf.predict(X_test)
    test_score.extend([clf.score(X_test, Y_test)])
    train_score.extend([clf.score(X_train, Y_train)])
print('train score :', train_score)
print('test score :', test_score)
print(np.mean(test_score))